**Verificação Caminho**

In [0]:
dbutils.fs.ls('/Workspace/csv/')

Nesta etapa, foram carregados os arquivos CSV originais e gravados em tabelas Delta sem qualquer modificação no conteúdo original (somente metadados adicionados). Cada tabela Bronze contém:

- Dados brutos, como estão no CSV
- Data de ingestão (ingestao_dt)
- Fonte de origem (source)
- Colunas com nomes padronizados em snake_case
- Armazenamento em formato Delta Lake
    - Melhor performance
    - Suporte a versionamento
    - Leitura e escrita otimizadas

Essas tabelas Bronze servem como única fonte de verdade para a transformação Silver.

In [0]:
from pyspark.sql import functions as F

# ======================================================================
# 1. Definir caminhos dos CSVs dentro do Workspace
# ======================================================================

path_full = '/Workspace/csv/campeonato-brasileiro-full.csv'
path_cartoes = '/Workspace/csv/campeonato-brasileiro-cartoes.csv'
path_gols = '/Workspace/csv/campeonato-brasileiro-gols.csv'
path_estatisticas = '/Workspace/csv/campeonato-brasileiro-estatisticas-full.csv'


# ======================================================================
# 2. Função utilitária para criar a tabela Bronze corretamente
# ======================================================================

def create_bronze_table(df, table_name, source_path):
    """
    - Adiciona colunas de metadados (ingestao_dt e source)
    - Padroniza nome das colunas
    - Salva como tabela Delta (bronze)
    """
    
    # 2.1 Data de ingestão
    df_bronze = df.withColumn("ingestao_dt", F.current_timestamp())
    
    # 2.2 Nome da origem (para documentação/linhagem)
    df_bronze = df_bronze.withColumn("source", F.lit(source_path))
    
    # 2.3 Padronizar nome das colunas para snake_case
    df_bronze = df_bronze.toDF(*[
        c.lower()
         .replace(" ", "_")
         .replace("-", "_")
         .replace("ç", "c")
         .replace("ã", "a")
         .replace("á", "a")
        for c in df_bronze.columns
    ])
    
    # 2.4 Salvar como Delta (tabela Bronze)
    df_bronze.write.format("delta") \
        .mode("overwrite") \
        .option("mergeSchema", "true") \
        .saveAsTable(table_name)
    
    print(f"Tabela criada: {table_name}")


# ======================================================================
# 3. Ler arquivos CSV brutos
# ======================================================================

df_full = spark.read.csv(path_full, header=True, inferSchema=True)
df_cartoes = spark.read.csv(path_cartoes, header=True, inferSchema=True)
df_gols = spark.read.csv(path_gols, header=True, inferSchema=True)
df_estatisticas = spark.read.csv(path_estatisticas, header=True, inferSchema=True)


# ======================================================================
# 4. Criar tabelas Bronze
# ======================================================================

create_bronze_table(df_full, "bronze_full", path_full)
create_bronze_table(df_cartoes, "bronze_cartoes", path_cartoes)
create_bronze_table(df_gols, "bronze_gols", path_gols)
create_bronze_table(df_estatisticas, "bronze_estatisticas", path_estatisticas)


# Mostrar as primeiras linhas de cada DataFrame
df_full.show(5)
df_cartoes.show(5)
df_gols.show(5)
df_estatisticas.show(5)